## Đồ án cuối kỳ - Lập trình cho Khoa học Dữ liệu

|Danh sách thành viên|MSSV|Phân chia công việc|Tiến độ hoàn thành|
|---|---|---|---|
|Đào Quốc Phong|18120505|||
|Hồ Hoàng Duy|19120207|||

## Thu thập dữ liệu

Dữ liệu trong đồ án lần này là về Phân tích Tính cách của Khách hàng (link [kaggle](https://www.kaggle.com/imakash3011/customer-personality-analysis)). Đây là một phân tích chi tiết về những khách hàng lý tưởng của một công ty. Nó giúp doanh nghiệp hiểu rõ hơn về khách hàng của mình và giúp họ dễ dàng sửa đổi sản phẩm theo nhu cầu, hành vi và mối quan tâm cụ thể của các loại khách hàng khác nhau.

Thông qua việc phân tích tính cách khách hàng giúp doanh nghiệp sửa đổi sản phẩm của mình dựa trên khách hàng mục tiêu từ các loại phân khúc khách hàng khác nhau. Ví dụ: thay vì chi tiền để tiếp thị một sản phẩm mới cho mọi khách hàng trong cơ sở dữ liệu của công ty, một công ty có thể phân tích phân khúc khách hàng nào có nhiều khả năng mua sản phẩm nhất và sau đó chỉ tiếp thị sản phẩm trên phân khúc cụ thể đó.

## Import các thư viện cơ bản

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Khám phá dữ liệu
### Đọc dữ liệu đầu vào

In [2]:
customer_df = pd.read_csv("marketing_campaign.csv", sep="\t")
customer_df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0


### Bộ dữ liệu có bao nhiêu hàng và cột?

In [3]:
num_rows, num_cols = customer_df.shape
print('Dữ liệu có {} dòng và {} cột'.format(num_rows, num_cols))

Dữ liệu có 2240 dòng và 29 cột


### Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

Mỗi dòng là chi tiết thông tin của một khách hàng được thu thập. Có vẻ không có dòng nào có ý nghĩa khác.

### Dữ liệu có dòng nào bị trùng không?

In [4]:
duplicated_rows = customer_df.duplicated()
num_duplicated_rows = len(customer_df[duplicated_rows])
print("Có {} dòng bị lặp.".format(num_duplicated_rows))

Có 0 dòng bị lặp.


### Mỗi cột có ý nghĩa gì?

Dưới đây là mô tả ý nghĩa của các cột trong file `marketing_campaign.csv`:   

*People*

ID: Customer's unique identifier  
Year_Birth: Customer's birth year  
Education: Customer's education level  
Marital_Status: Customer's marital status  
Income: Customer's yearly household income  
Kidhome: Number of children in customer's household  
Teenhome: Number of teenagers in customer's household  
Dt_Customer: Date of customer's enrollment with the company  
Recency: Number of days since customer's last purchase  
Complain: 1 if the customer complained in the last 2 years, 0 otherwise  

*Products*

MntWines: Amount spent on wine in last 2 years  
MntFruits: Amount spent on fruits in last 2 years  
MntMeatProducts: Amount spent on meat in last 2 years  
MntFishProducts: Amount spent on fish in last 2 years  
MntSweetProducts: Amount spent on sweets in last 2 years  
MntGoldProds: Amount spent on gold in last 2 years  

*Promotion*

NumDealsPurchases: Number of purchases made with a discount  
AcceptedCmp1: 1 if customer accepted the offer in the 1st campaign, 0 otherwise  
AcceptedCmp2: 1 if customer accepted the offer in the 2nd campaign, 0 otherwise  
AcceptedCmp3: 1 if customer accepted the offer in the 3rd campaign, 0 otherwise  
AcceptedCmp4: 1 if customer accepted the offer in the 4th campaign, 0 otherwise  
AcceptedCmp5: 1 if customer accepted the offer in the 5th campaign, 0 otherwise  
Response: 1 if customer accepted the offer in the last campaign, 0 otherwise  

*Place*

NumWebPurchases: Number of purchases made through the company’s website  
NumCatalogPurchases: Number of purchases made using a catalogue  
NumStorePurchases: Number of purchases made directly in stores  
NumWebVisitsMonth: Number of visits to company’s website in the last month  

### Mỗi cột hiện đang có kiểu dữ liệu gì?

In [5]:
col_dtypes = customer_df.dtypes
col_dtypes

ID                       int64
Year_Birth               int64
Education               object
Marital_Status          object
Income                 float64
Kidhome                  int64
Teenhome                 int64
Dt_Customer             object
Recency                  int64
MntWines                 int64
MntFruits                int64
MntMeatProducts          int64
MntFishProducts          int64
MntSweetProducts         int64
MntGoldProds             int64
NumDealsPurchases        int64
NumWebPurchases          int64
NumCatalogPurchases      int64
NumStorePurchases        int64
NumWebVisitsMonth        int64
AcceptedCmp3             int64
AcceptedCmp4             int64
AcceptedCmp5             int64
AcceptedCmp1             int64
AcceptedCmp2             int64
Complain                 int64
Z_CostContact            int64
Z_Revenue                int64
Response                 int64
dtype: object

Ta thấy có 3 cột có kiểu dữ liệu là `object` là: `Education`, `Marital_Status`, `Dt_Customer`. Quan sát vài bộ dữ liệu thì ta có thể đoán cột `Education` và `Marital_Status` có kiểu dữ liệu `str` và thuộc dạng categorical, cột `Dt_Customer` có dạng "Ngày tháng năm". Các cột còn lại có dạng numerical

## Đưa ra các câu hỏi có ý nghĩa để trả lời

## Tiền xử lý + Phân tích dữ liệu để trả lời các câu hỏi trên

## Nhìn lại quá trình làm đồ án

## Tài liệu tham khảo